<a href="https://colab.research.google.com/github/bmreiniger/datascience.stackexchange/blob/master/57905.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sklearn
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
cancer=load_breast_cancer()

X_train, X_test, y_train, y_test=sklearn.model_selection.train_test_split(cancer.data, 
                                                                         cancer.target, 
                                                                         random_state=0)
xg_clas=xgb.XGBClassifier(random_state=1, n_estimators=100).fit(X_train, y_train)
print(xg_clas.score(X_train, y_train),"\n", xg_clas.score(X_test, y_test))

1.0 
 0.9790209790209791


In [3]:
xg_clas

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [4]:
X_test[0,:].reshape(1,-1)

array([[1.340e+01, 2.052e+01, 8.864e+01, 5.567e+02, 1.106e-01, 1.469e-01,
        1.445e-01, 8.172e-02, 2.116e-01, 7.325e-02, 3.906e-01, 9.306e-01,
        3.093e+00, 3.367e+01, 5.414e-03, 2.265e-02, 3.452e-02, 1.334e-02,
        1.705e-02, 4.005e-03, 1.641e+01, 2.966e+01, 1.133e+02, 8.444e+02,
        1.574e-01, 3.856e-01, 5.106e-01, 2.051e-01, 3.585e-01, 1.109e-01]])

In [5]:
dump_list=xg_clas.get_booster().get_dump()
num_t=len(dump_list)

preds = [0]
for n in range(num_t):
  preds.append(xg_clas.predict(X_test[0,:].reshape(1,-1), ntree_limit=n+1, output_margin=True))

print([preds[i+1]-preds[i] for i in range(num_t)])

[array([-0.19117647], dtype=float32), array([-0.17493302], dtype=float32), array([-0.1632322], dtype=float32), array([-0.15528071], dtype=float32), array([-0.14764059], dtype=float32), array([-0.134592], dtype=float32), array([-0.10049492], dtype=float32), array([-0.1302135], dtype=float32), array([-0.1268928], dtype=float32), array([-0.11612129], dtype=float32), array([-0.08275878], dtype=float32), array([-0.1172291], dtype=float32), array([-0.11398101], dtype=float32), array([-0.1115346], dtype=float32), array([-0.11064661], dtype=float32), array([-0.07345426], dtype=float32), array([-0.10717297], dtype=float32), array([-0.10513449], dtype=float32), array([-0.10398841], dtype=float32), array([-0.10136986], dtype=float32), array([-0.06175613], dtype=float32), array([-0.0963707], dtype=float32), array([-0.07870603], dtype=float32), array([-0.09415579], dtype=float32), array([-0.09520769], dtype=float32), array([-0.09691024], dtype=float32), array([-0.08932781], dtype=float32), array([0

In [6]:
sum([preds[i+1]-preds[i] for i in range(num_t)])

array([-5.9249835], dtype=float32)

In [7]:
xg_clas.predict(X_test[:10,:], ntree_limit=0, output_margin=True)

array([-5.9249835,  3.9154263,  6.179787 ,  5.1894045,  5.766832 ,
        6.7987423,  6.1742053,  5.7832847,  5.014198 ,  7.2340755],
      dtype=float32)

In [8]:
print(dump_list[0])

0:[f7<0.0489199981] yes=1,no=2,missing=1
	1:[f20<16.8250008] yes=3,no=4,missing=3
		3:[f10<0.591250002] yes=7,no=8,missing=7
			7:leaf=0.191869915
			8:leaf=-0
		4:[f1<18.9599991] yes=9,no=10,missing=9
			9:leaf=0.120000005
			10:leaf=-0.13333334
	2:[f23<785.799988] yes=5,no=6,missing=5
		5:[f21<23.7399998] yes=11,no=12,missing=11
			11:leaf=0.155555561
			12:leaf=-0.100000001
		6:[f1<14.3000002] yes=13,no=14,missing=13
			13:leaf=-0
			14:leaf=-0.191176474



`X_test[0,:]` appears to land in leaf 14 of the first tree, which checks out with the above predictions.